In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, './venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

# Load a Pretrained Model

In [ ]:
# pretrained model options:
# - a 0.5m parameter model trained for 500 epochs: 'long_and_skinny'
# - a 0.5m parameter model trained for 500 epochs: 'medium'
# - a 0.5m parameter model trained for 500 epochs: 'short_and_thick'
name = 'short_and_thick'

from tools import load_model
model, tokenizer, cfg = load_model(name)

# If you only plan to do inference, switch to evaluation mode
model.eval()

# Inference

In [5]:
from inference import generate
prompt = "Once upon a time"
output = generate(
    prompt, 
    model, 
    tokenizer,
    #max_gen_len = 100,
    temperature = 0.7,
    memory_saver_div = 8,
    top_p = 0.9,
    top_k = 32,
)
print(output)

maximum attention matrix size in memory will be 32x256 rather than 256x256

Once upon a time fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fast fa